In [ ]:
import cv2
import numpy as np
import math
import time
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Initialize hand detection
cap = cv2.VideoCapture(0)
hand_detector = HandDetector(maxHands=1)
hand_classifier = Classifier(r"C:\Users\ihebm\Desktop\sign detection\signG3.h5", r"C:\Users\ihebm\Desktop\sign detection\labels.txt")
hand_labels = ["Hello", "Bye", "Okay", "Thanks", "Yes", "No"]
offset = 20
imgSize = 300

# Initialize emotion detection
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
emotion_classifier = load_model(r'final_emotion_modelG1.h5')
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

emotion_start_time = time.time()
last_dominant_emotion = 'Neutral'

def draw_progress_bar(img, emotion, value, x, y, width=200, height=20):
    cv2.rectangle(img, (x, y), (x + width, y + height), (255, 255, 255), 1)
    filled_width = int(width * value)
    cv2.rectangle(img, (x, y), (x + filled_width, y + height), (0, 255, 0), -1)
    
    text = f"{emotion}: {value:.2f}"
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    text_x = x + (width - text_width) // 2
    text_y = y + height - (height - text_height) // 2
    
    cv2.putText(img, text, (text_x - 1, text_y - 1), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(img, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        continue

    imgOutput = img.copy()
    
    # Hand sign detection
    hands, img = hand_detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']
        h_img, w_img, _ = img.shape
        if x < 0 or y < 0 or x + w > w_img or y + h > h_img:
            print(f"Invalid bounding box: x={x}, y={y}, w={w}, h={h}")
        else:
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            imgCrop = img[y-offset:y + h + offset, x-offset:x + w + offset]
            if imgCrop.size > 0:
                imgCropShape = imgCrop.shape
                aspectRatio = h / w
                if aspectRatio > 1:
                    k = imgSize / h
                    wCal = math.ceil(k * w)
                    imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                    imgResizeShape = imgResize.shape
                    wGap = math.ceil((imgSize - wCal) / 2)
                    imgWhite[:, wGap:wCal + wGap] = imgResize
                else:
                    k = imgSize / w
                    hCal = math.ceil(k * h)
                    imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                    imgResizeShape = imgResize.shape
                    hGap = math.ceil((imgSize - hCal) / 2)
                    imgWhite[hGap:hCal + hGap, :] = imgResize
                
                prediction, index = hand_classifier.getPrediction(imgWhite, draw=False)
                cv2.rectangle(imgOutput, (x-offset, y-offset-70), (x-offset+400, y-offset+60-50), (0, 255, 0), cv2.FILLED)
                cv2.putText(imgOutput, hand_labels[index], (x, y-30), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)
                cv2.rectangle(imgOutput, (x-offset, y-offset), (x + w + offset, y + h + offset), (0, 255, 0), 4)

    # Emotion detection
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(imgOutput, (x,y), (x+w,y+h), (0,255,255), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48,48), interpolation=cv2.INTER_AREA)
        
        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)
            prediction = emotion_classifier.predict(roi)[0]
            label = emotion_labels[prediction.argmax()]
            label_position = (x, y)
            cv2.putText(imgOutput, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
            
            # Draw progress bars
            for i, (emotion, value) in enumerate(zip(emotion_labels, prediction)):
                draw_progress_bar(imgOutput, emotion, value, 10, 50 + i * 25)
            
            # Update dominant emotion
            dominant_emotion = emotion_labels[prediction.argmax()]
            if dominant_emotion != last_dominant_emotion:
                emotion_start_time = time.time()
                last_dominant_emotion = dominant_emotion
            
            # Draw emotion indicator circle
            circle_x = 30
            circle_y = imgOutput.shape[0] - 30
            circle_color = (255, 255, 255)  # Default white
            circle_radius = 10  # Initial size
            emotion_duration = time.time() - emotion_start_time
            if emotion_duration > 5:
                circle_radius = 15  # Scale up after 5 seconds
                if dominant_emotion == 'Happy':
                    circle_color = (0, 255, 0)  # Green
                elif dominant_emotion == 'Sad':
                    circle_color = (0, 165, 255)  # Orange
                elif dominant_emotion in ['Fear', 'Angry']:
                    circle_color = (0, 0, 255)  # Red
                elif dominant_emotion in ['Surprise', 'Disgust']:
                    circle_color = (0, 255, 255)  # Yellow
            if emotion_duration > 60 and dominant_emotion in ['Sad', 'Neutral']:
                circle_color = (255, 0, 0)  # Blue
                cv2.putText(imgOutput, "Autism risk", (circle_x - 30, circle_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.circle(imgOutput, (circle_x, circle_y), circle_radius, circle_color, -1)
        else:
            cv2.putText(imgOutput, 'No Faces', (30,80), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow('Hand Sign and Emotion Detector', imgOutput)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()